In [ ]:
# warning filters
import warnings
warnings.filterwarnings("ignore", message="Pandas requires version")
warnings.filterwarnings("ignore", message="A NumPy version >=")

# general imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# anonymity library
import pycanon
from anjana.anonymity import k_anonymity, l_diversity, t_closeness

# ML models
from xgboost import XGBClassifier as XGB

# our generic functions
from utils import get_metrics, write_suppression_results_to_csv, get_generalization_levels, get_train_test_data

# our data-specific functions
from utils import clean_process_data, get_hierarchies
import config_experiments as cfg

# Main execution
write_suppression_results_to_csv([], header=True)

# Define the parameters
dataset = 'adult' # 'bank', 'adult'

# Get parameters from config file
lst_supp_level = cfg.supp_level
dic_methods_parameters = {
                            'k-anonymity': cfg.fixed_k,
                            'l-diversity': cfg.fixed_l,
                            't-closeness': cfg.fixed_t
                        } 
max_seed = cfg.max_seed
test_size = cfg.test_size
if dataset == 'adult':
    lst_threshold_target = cfg.adult_threshold_target
else:
    lst_threshold_target = [None]

# Loop over the three anonymization methods
for method, anon_parameter in dic_methods_parameters.items():
    print(f"Method: {method}, Parameter: {anon_parameter}") 

    # Loop over several threshold targets (same dataset but with different Y distribution)
    for threshold_target in lst_threshold_target:
        print(f"Threshold target: {threshold_target}")

        # read data
        if dataset == 'adult':

            # Sensitive/target and protected attributes
            sens_att = "income"
            protected_att = "gender"
            
            # Read and process the data
            data = clean_process_data(pd.read_csv("adult_reconstruction.csv"), dataset, sens_att, protected_att, threshold_target)

        elif dataset == 'bank':

            # Sensitive/target and protected attributes
            sens_att = "y"
            protected_att = "age"
            
            # Read and process the data
            data = clean_process_data(pd.read_csv("bank-full.csv", delimiter=';'), dataset, sens_att, protected_att, threshold_target)

        # Import/defining the hierarquies for each quasi-identifier. 
        hierarchies = get_hierarchies(data, dataset)

        # Define the quasi-identifiers and the sensitive/protected attribute
        quasi_ident = list(set(data.columns) - {protected_att} - {sens_att})

        # Loop over several seeds
        SEED = 0
        while SEED < max_seed:
            print(f"SEED: {SEED}")

            # Loop over several suppression values
            for supp_level in lst_supp_level:
                print(f"supp_level: {supp_level}")

                try:
                    # Split into train and test data
                    train_data, test_data = train_test_split(data, test_size=test_size, random_state=SEED)

                    # Anonymize data
                    train_data_anon = k_anonymity(train_data, [], quasi_ident, cfg.fixed_k, supp_level, hierarchies)
                    if 'index' in train_data_anon.columns:
                        del train_data_anon['index'] 

                    # Assert that the level of k-anonymity is at least k
                    actual_k_anonymity = pycanon.anonymity.k_anonymity(train_data_anon, quasi_ident)
                    assert actual_k_anonymity >= cfg.fixed_k, f"k-anonymity constraint not met: Expected >= {cfg.fixed_k}, but got {actual_k_anonymity}"

                    if method == 'l-diversity':
                        # Apply l-diversity
                        train_data_anon = l_diversity(train_data_anon, [], quasi_ident, sens_att, cfg.fixed_k, anon_parameter, supp_level, hierarchies)

                        # Assert that the level of l-diversity is exactly met
                        actual_l_diversity = pycanon.anonymity.l_diversity(train_data_anon, quasi_ident, [sens_att])
                        assert actual_l_diversity == anon_parameter, f"l-diversity constraint not met: Expected == {anon_parameter}, but got {actual_l_diversity}"

                    if method == 't-closeness':
                        # Apply t-closeness
                        train_data_anon = t_closeness(train_data_anon, [], quasi_ident, sens_att, cfg.fixed_k, anon_parameter, supp_level, hierarchies)

                        # Assert that the level of t-closeness is satisfied
                        actual_t_closeness = pycanon.anonymity.t_closeness(train_data_anon, quasi_ident, [sens_att], True)
                        assert actual_t_closeness <= anon_parameter, f"t-closeness constraint not met: Expected <= {anon_parameter}, but got {actual_t_closeness:.2f}"

                    # Get generalization levels of the training set to apply the same to the test set
                    generalization_levels = get_generalization_levels(train_data_anon, quasi_ident, hierarchies)

                    # Apply the same generalization levels to the test data (Except for the protected attribute: for fairness measurements)
                    for col in set(quasi_ident) - {protected_att}:
                        level = generalization_levels.get(col)
                        
                        if level is not None:
                            # Retrieve the mapping dictionary for this level
                            hierarchy_mapping = dict(zip(hierarchies[col][0], hierarchies[col][level]))
                            
                            # Apply the mapping to the test data
                            test_data[col] = test_data[col].map(hierarchy_mapping)

                    # Separate features and target
                    X_train, y_train, X_test, y_test = get_train_test_data(train_data_anon, test_data, sens_att)

                    # Train the model
                    model = XGB(random_state=SEED, n_jobs=-1)
                    model.fit(X_train, y_train)

                    # Get fairness/utility metrics
                    df_fm = test_data.copy()
                    df_fm['y_pred'] = np.round(model.predict(X_test)).reshape(-1).astype(int)
                    dic_metrics = get_metrics(df_fm, protected_att, sens_att)
                    print(dic_metrics)

                    # Write results to csv
                    write_suppression_results_to_csv([SEED, dataset + "_" + str(threshold_target), protected_att, sens_att, method, anon_parameter, supp_level] + list(dic_metrics.values()))

                except Exception as e:
                        print(f"An error occurred for SEED {SEED}, k {cfg.fixed_k}: {e}")
                        continue
            
            SEED += 1
            print('-------------------------------------------------------------\n')
        print('=============================================================\n')
    print('#############################################################\n')

Method: k-anonymity, Parameter: 10
Threshold target: None
SEED: 0
supp_level: 10
{'SPD': -0.11103368776022848, 'EOD': -0.14049586776859505, 'MAD': 0.2747984125345537, 'PED': -0.09350957636660845, 'PRD': -0.4722222222222222, 'ACC': 0.8820406959598939, 'f1': 0.040767386091127095, 'Precision': 0.4657534246575342, 'Recall': 0.021316614420062698, 'ROC_AUC': 0.5090290984206588, 'CM': array([[11930,    39],
       [ 1561,    34]], dtype=int64)}
supp_level: 20
{'SPD': -0.17359653449258075, 'EOD': -0.1697574413760667, 'MAD': 0.2312998711522507, 'PED': -0.11376762724998926, 'PRD': -0.00024245095878328016, 'ACC': 0.890002948982601, 'f1': 0.2806171648987464, 'Precision': 0.6075156576200418, 'Recall': 0.18244514106583073, 'ROC_AUC': 0.5833689486764528, 'CM': array([[11781,   188],
       [ 1304,   291]], dtype=int64)}
supp_level: 30
{'SPD': -0.20168926281499835, 'EOD': -0.21390176711684475, 'MAD': 0.23237854092349808, 'PED': -0.13671345965494663, 'PRD': 0.014317111459968612, 'ACC': 0.88948687702742

c:\Users\heber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'SPD': -0.1476510067114094, 'EOD': -0.1703056768558952, 'MAD': 0.29659585391402266, 'PED': -0.1335149863760218, 'PRD': -0.4431818181818182, 'ACC': 0.8825567679150693, 'f1': 0.0466786355475763, 'Precision': 0.4431818181818182, 'Recall': 0.024636765634870498, 'ROC_AUC': 0.5102734783854179, 'CM': array([[11932,    49],
       [ 1544,    39]], dtype=int64)}
supp_level: 20
{'SPD': -0.17514252946851103, 'EOD': -0.1834286893758103, 'MAD': 0.23092568594247476, 'PED': -0.1070921639719797, 'PRD': -0.006579990486760723, 'ACC': 0.8919197876732528, 'f1': 0.2742574257425743, 'Precision': 0.6338672768878718, 'Recall': 0.1749842072015161, 'ROC_AUC': 0.5808148646390687, 'CM': array([[11821,   160],
       [ 1306,   277]], dtype=int64)}
supp_level: 30
{'SPD': -0.1536389005047013, 'EOD': -0.10119780949862286, 'MAD': 0.2730262204538586, 'PED': -0.12369924675143079, 'PRD': 0.09920682610263187, 'ACC': 0.8902241226776763, 'f1': 0.25661507738392414, 'Precision': 0.611904761904762, 'Recall': 0.162349968414403

c:\Users\heber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'SPD': -0.0910569105691057, 'EOD': -0.09859154929577464, 'MAD': 0.26129813835013793, 'PED': -0.08706467661691543, 'PRD': -0.375, 'ACC': 0.8803450309643173, 'f1': 0.025225225225225224, 'Precision': 0.375, 'Recall': 0.01305158483530143, 'ROC_AUC': 0.5050619697493111, 'CM': array([[11920,    35],
       [ 1588,    21]], dtype=int64)}
supp_level: 20
{'SPD': -0.17520617155356819, 'EOD': -0.18974736672182088, 'MAD': 0.21156532161506647, 'PED': -0.11632825227278307, 'PRD': 0.09983606557377045, 'ACC': 0.8914037157180773, 'f1': 0.27474150664697194, 'Precision': 0.6611374407582938, 'Recall': 0.17339962709757614, 'ROC_AUC': 0.5807190523610005, 'CM': array([[11812,   143],
       [ 1330,   279]], dtype=int64)}
supp_level: 30
{'SPD': -0.13030406340823003, 'EOD': -0.08836783835774917, 'MAD': 0.2310775167370177, 'PED': -0.09744383767994615, 'PRD': 0.11857534930139713, 'ACC': 0.8905190209377765, 'f1': 0.27170181461500736, 'Precision': 0.6441860465116279, 'Recall': 0.17215661901802362, 'ROC_AUC': 0.57

...

c:\Users\heber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'SPD': 0.0, 'EOD': 0.0, 'MAD': 0.2614661015148617, 'PED': 0.0, 'PRD': 0.0, 'ACC': 0.8847685048658213, 'f1': 0.0, 'Precision': 0.0, 'Recall': 0.0, 'ROC_AUC': 0.5, 'CM': array([[12001,     0],
       [ 1563,     0]], dtype=int64)}
supp_level: 50
The data verifies k-anonymity with k=10


c:\Users\heber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'SPD': 0.0, 'EOD': 0.0, 'MAD': 0.2614661015148617, 'PED': 0.0, 'PRD': 0.0, 'ACC': 0.8847685048658213, 'f1': 0.0, 'Precision': 0.0, 'Recall': 0.0, 'ROC_AUC': 0.5, 'CM': array([[12001,     0],
       [ 1563,     0]], dtype=int64)}
-------------------------------------------------------------

SEED: 34
supp_level: 10
The data verifies k-anonymity with k=10


KeyboardInterrupt: 